### Week 1 notebook

Each week, you will apply the concepts of that week to your Integrated Capstone Project’s dataset. In preparation for Milestone One, create a Jupyter Notebook (similar to in Module B, semester two) that illustrates these lessons. There are no specific questions to answer in your Jupyter Notebook files in this course; your general goal is to analyze your data, using the methods you have learned about in this course and in this program, and draw interesting conclusions. 

For Week 1, include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday. 

In Week 7, you will compile your findings from your Jupyter Notebook homework into your Milestone One assignment for grading. For full instructions and the rubric for Milestone One, refer to the following link. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys


In [2]:
# Analyzing the carclaims_cleaned dataset
df_car = pd.read_csv('C:/Users/JT von Seggern/DS Masters Repos/2025-spring-B2/data/carclaims_cleaned.csv')
pd.set_option('display.max_columns', None)
df_car.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,RepNumber,Deductible,DriverRating,Days:Policy-Accident,Days:Policy-Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,FraudFound
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,"more than 69,000",12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,No
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,"more than 69,000",15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,No
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,"more than 69,000",7,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,No
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,"20,000 to 29,000",4,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability,No
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,"more than 69,000",3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision,No


In [3]:
# encode categorical features
def encode_categories(df: pd.DataFrame, cat_cols: list) -> tuple[pd.DataFrame, dict]:
    df_encoded = df.copy()
    mappings = {}

    for col in cat_cols:
        # get the unique categories (in order of appearance)
        uniques = df_encoded[col].dropna().unique().tolist()
        # build a mapping: category -> code
        mapping = {cat: code for code, cat in enumerate(uniques)}
        mappings[col] = mapping

        # map (leave NaN as NaN)
        df_encoded[col] = df_encoded[col].map(mapping).astype("int64")

    return df_encoded, mappings


In [4]:
# Convert categorical features to dummy variables
car_dum = df_car.copy()
cat_feat = car_dum.select_dtypes(include=['object']).columns
car_encode, map = encode_categories(car_dum, cat_feat)

In [5]:
# Split data into features and targets
X_car_encode = car_encode.drop(columns=['FraudFound'])
y_car_encode = car_encode['FraudFound']

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate vif for each feature
vif_car_data = pd.DataFrame()
vif_car_data['Feature'] = X_car_encode.columns
vif_car_data['VIF'] = [variance_inflation_factor(X_car_encode.values, i) for i in range(len(X_car_encode.columns))]

# Sort the features by vif
vif_car_data = vif_car_data.sort_values(by='VIF', ascending=False)

# Display the vif data
vif_car_data

,Feature,VIF
29,Year,174.967540
12,PolicyType,116.822077
16,Deductible,91.617858
13,VehicleCategory,74.768882
28,NumberOfCars,17.599866
30,BasePolicy,14.921937
10,Age,13.456099
27,AddressChange-Claim,12.508655
8,Sex,6.494825
1,WeekOfMonth,6.179695


In [7]:
# Create a linear regression model using the info fro the vif analysis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Only pick columns with a vif < 10
car_cols = list(vif_car_data[vif_car_data['VIF'] < 10]['Feature'])

# Split the data into train and test sets
X_car_train, X_car_test, y_car_train, y_car_test = train_test_split(X_car_encode[car_cols], y_car_encode, test_size=.2, random_state=0)

In [8]:
#Scale the data
from sklearn.preprocessing import StandardScaler
scaler_car = StandardScaler()
X_car_train_scaled = scaler_car.fit_transform(X_car_train)
X_car_test_scaled = scaler_car.fit_transform(X_car_test)

In [9]:
X_car_encode[car_cols].head()

,Sex,WeekOfMonth,WeekOfMonthClaimed,DriverRating,AgeOfVehicle,RepNumber,Month,MonthClaimed,MaritalStatus,AgeOfPolicyHolder,DayOfWeek,VehiclePrice,PastNumberOfClaims,DayOfWeekClaimed,Make,NumberOfSuppliments,Days:Policy-Accident,Days:Policy-Claim,Fault,AccidentArea,PoliceReportFiled,WitnessPresent,AgentType
0,0,5,1,1,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,3,4,4,1,15,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
2,1,5,2,3,2,7,2,1,1,2,1,0,1,2,0,0,0,0,0,0,0,0,0
3,1,2,1,2,3,4,3,2,1,3,2,1,1,3,1,1,0,0,1,1,1,0,0
4,0,5,2,1,4,3,1,3,0,1,3,0,0,0,0,0,0,0,1,0,0,0,0


In [10]:
# Create the logistic regression model
lm_car = LinearRegression().fit(X_car_train_scaled, y_car_train)

# Make predictions
y_car_pred = lm_car.predict(X_car_test_scaled)

# Score the model
score_car = lm_car.score(X_car_test_scaled, y_car_test)

# Display the results
print(f"Model Score: {score_car}")

Model Score: 0.030695881519799295


In [11]:
# Load the insurance claims dataset
df_ins = pd.read_csv("C:/Users/JT von Seggern/DS Masters Repos/2025-spring-B2/data/insurance_claims_cleaned.csv")
df_ins.head()

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,1
1,228,42,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,1
2,134,29,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,0
3,256,41,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,1
4,228,44,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,Vehicle Theft,?,Minor Damage,No authorities contacted,NY,Arlington,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,0


In [12]:
# Split the data into features and targets
X_ins = df_ins.drop(columns=['fraud_reported'])
y_ins = df_ins['fraud_reported']

In [13]:
# Encode all categorical features
ins_cat_feat = X_ins.select_dtypes(include=['object']).columns
X_ins_encoded, ins_map = encode_categories(X_ins, ins_cat_feat)

In [14]:
# Calculate vif for each feature
vif_ins_data = pd.DataFrame()
vif_ins_data['Feature'] = X_ins_encoded.columns
vif_ins_data['VIF'] = [variance_inflation_factor(X_ins_encoded.values, i) for i in range(len(X_ins_encoded.columns))]

# Sort the features by vif
vif_ins_data = vif_ins_data.sort_values(by='VIF', ascending=False)

# Display the vif data
vif_ins_data

c:\Users\JT von Seggern\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,Feature,VIF
28,injury_claim,inf
27,total_claim_amount,inf
29,property_claim,inf
30,vehicle_claim,inf
33,auto_year,166.416587
1,age,131.686138
7,insured_zip,50.881748
0,months_as_customer,28.250718
5,policy_annual_premium,28.077538
22,number_of_vehicles_involved,13.428271


In [15]:
# Only pick columns with a vif < 10
ins_cols = list(vif_ins_data[vif_ins_data['VIF'] < 10]['Feature'])

# Split the data into train and test sets
X_ins_train, X_ins_test, y_ins_train, y_ins_test = train_test_split(X_ins_encoded[ins_cols], y_ins, test_size=.2, random_state=0)

In [16]:
#Scale the data
scaler_ins = StandardScaler()
X_ins_train_scaled = scaler_ins.fit_transform(X_ins_train)
X_ins_test_scaled = scaler_ins.fit_transform(X_ins_test)

In [18]:
# Create the logistic regression model
lm_ins = LinearRegression().fit(X_ins_train_scaled, y_ins_train)

# Make predictions
y_ins_pred = lm_ins.predict(X_ins_test_scaled)

# Score the model
score_ins = lm_ins.score(X_ins_test_scaled, y_ins_test)

# Display the results
print(f"Model Score: {score_ins}")

Model Score: 0.1549977101155403
